In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools
import pathlib
import pyam

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# these functions can be found in utils.py in this same directory
from utils import make_quantiles

sns.set_style('whitegrid')

In [3]:
plt.rcParams["pdf.fonttype"] = 42

# Load Data

In [4]:
data_path = pathlib.Path('../data')

In [5]:
paper_data = pd.concat([
    pd.read_csv(data_path / 'gidden_et_al_2022_ar6_reanalysis_data.csv'),
    pd.read_csv(data_path / 'additional_analysis_data.csv'),
])

In [6]:
ar6_data = pd.read_csv(data_path / 'AR6_Scenarios_Database_World_v1.0.csv')
ar6_data = ar6_data[ar6_data.Variable.isin(['Emissions|CO2|AFOLU'])]

In [7]:
df = pyam.IamDataFrame(pd.concat([paper_data, ar6_data]))
df.load_meta(data_path / 'gidden_et_al_2022_ar6_reanalysis_meta.csv')

In [8]:
categories_to_temp = {'C1': '1.5C', 'C3': '2C'}
df.meta['Temperature'] = df.meta['Category'].replace(categories_to_temp)

In [9]:
df.variable

# Panel 1A

In [10]:
variables = [
    'Emissions|CO2|AFOLU',
    'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU', 
    'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct', 
]

data = pyam.concat([
    make_quantiles(df, v, cat)
    for v, cat in itertools.product(variables, categories_to_temp.keys())
])

In [11]:
def afolu_line_plot(data, variables, cat, ax=None, years=range(1990, 2051), color='variable'):
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))
    (
        data
        .filter(variable=variables, year=years, model=cat)
        .filter(scenario=['*0.25*', '*0.75*'])
        .convert_unit('Mt CO2/yr', to='Gt CO2/yr', factor=1e-3)
        .plot
        .line(color=color, fill_between=True, ax=ax, alpha=0.1, legend=False) #  cmap='PiYG_r',
    )
    (
        data
        .filter(variable=variables, year=years, model=cat)
        .filter(scenario=['*0.5*'])
        .convert_unit('Mt CO2/yr', to='Gt CO2/yr', factor=1e-3)
        .plot
        .line(color=color, ax=ax)  # cmap='PiYG_r',
    )
    h, l = ax.get_legend_handles_labels()
    ax.legend(h[-len(variables):], l[-len(variables):])
    return ax

In [12]:
color_map = {
    'Emissions|CO2|AFOLU': '#' + 'c' * 6,
    'LULUCF Emissions - Model Reporting Convensions': '#a23247',
    'LULUCF Emissions - NGHGI Reporting Conventions': '#628C79', 
}
pyam.run_control().update({'color': {'variable': color_map}})

In [13]:
error = pd.read_excel(data_path / 'LUC_uncertainty_AR6_reanalysis.xlsx', sheet_name='Fig2AB')

In [14]:
def add_eluc_history(error, ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))
    colors = {
        'Model': '#a23247',
        'NGHGI': '#628C79', 
    }
    markers = {
        '2002-2011': 's',
        '2012-2021': 'o', 
        '2000-2020': 'd',
    }
    for i, row in error.iterrows():
        label = ' '.join((row['label'], row['emissions'], row['period']))
        ax.scatter(row['plot_year'], row['mean'], marker=markers[row['period']],  label=label, 
                   color=colors[row['emissions']], s=100, edgecolor='k', linewidths=2)
        ax.errorbar(row['plot_year'], row['mean'], row['std'], color=colors[row['emissions']])
    ax.legend()
    return ax

add_eluc_history(error)

In [15]:
cat = 'C1'

variables = {
    'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct': 'LULUCF Emissions - Model Reporting Convensions',
    'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU': 'LULUCF Emissions - NGHGI Reporting Conventions', 
}
fig, ax = plt.subplots(figsize=(9, 4))
afolu_line_plot(data.rename(variable=variables), variables.values(), cat, years=range(1990, 2101), ax=ax)
add_eluc_history(error, ax=ax)
ax.axhline(0, c='k', alpha=0.25)
ax.set_ylim([-8, 8])

ax.set_title(f'Interquartile Range of LULUCF CO2 Emissions in {categories_to_temp[cat]} Pathways')
ax.set_xlabel('')
h, l = ax.get_legend_handles_labels()
ax.legend(h[-7:], l[-7:], **pyam.plotting.OUTSIDE_LEGEND['right'])

fig.savefig('./files/fig2a1.pdf', dpi=1200, bbox_inches='tight')

In [16]:
cat = 'C3'

variables = {
    'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct': 'LULUCF Emissions - Model Reporting Convensions',
    'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU': 'LULUCF Emissions - NGHGI Reporting Conventions', 
}
fig, ax = plt.subplots(figsize=(9, 4))
afolu_line_plot(data.rename(variable=variables), variables.values(), cat, years=range(1990, 2101), ax=ax)
add_eluc_history(error, ax=ax)
ax.axhline(0, c='k', alpha=0.25)
ax.set_ylim([-8, 8])


ax.set_title(f'Interquartile Range of LULUCF CO2 Emissions in {categories_to_temp[cat]} Pathways')
ax.set_xlabel('')
ax.legend().remove()

fig.savefig('./files/fig2a2.pdf', dpi=1200, bbox_inches='tight')

# Panel 1B

In [17]:
error = pd.read_excel(data_path / 'LUC_uncertainty_AR6_reanalysis.xlsx', sheet_name='Fig2C')
error

In [18]:
fig, ax = plt.subplots(figsize=(1.5, 8))
markers = {
    '2000-2020': 's',
    '2000-2010': 'd',
    '2005-2015': 'o', 
    '2010-2020': '*',
}
colors = {
    'GCB (2022)': 'slategrey',
    'Grassi (2021)': 'royalblue', 
    'Grassi (2023)': 'cornflowerblue',
    'OSCAR (this study)': 'midnightblue',
}
for i, row in error.iterrows():
    label = ' '.join((row['label'], row['period']))
    ax.scatter(-1 + i, row['mean'], marker=markers[row['period']],  label=label, color=colors[row['label']], s=100)
    ax.errorbar(-1 + i, row['mean'], row['std'], color=colors[row['label']])

# set xticks
ax.set_xlim((-2, 2))
ax.set_xticks([])
ax.set_xlabel('2000-2020')

# conform with below plot
ax.set_ylabel('Gt CO2/yr')
plt.grid(axis='x', visible=False)

## This block should be commented/uncommented to get adobe-ready versions
#ax.legend(**pyam.plotting.OUTSIDE_LEGEND['right'])

# harmonize ylim
ax.set_ylim(-4.5, 9.5)

fig.savefig('./files/fig2b1.pdf', bbox_inches='tight')

In [19]:
color_map = {
    '1.5C': pyam.plotting.PYAM_COLORS["AR6-C1"],
    '1.5C-OS': pyam.plotting.PYAM_COLORS["AR6-C2"],
    '2C': pyam.plotting.PYAM_COLORS["AR6-C3"],
}
pyam.run_control().update({'color': {'Temperature': color_map}})

In [20]:
variables = [
    'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct', 
    'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU', 
]
name = 'NGHGI LULUCF Gap'
fig, ax = plt.subplots(figsize=(2, 8))
(
    df
    .filter(region='World', variable=variables)
    .filter(Temperature=['1.5C', '2C'])
    .filter(year=range(2025, 2101, 25))
    .subtract(variables[0], variables[1], name=name, ignore_units='Mt CO2/yr')
    .convert_unit('Mt CO2/yr', to='Gt CO2/yr', factor=1e-3)
    .plot
#    .box(color='thistle', by='Temperature', ax=ax, showfliers=False)
    .box(by='Temperature', ax=ax, showfliers=False)
)
ax.set_xlabel('')

## This block should be commented/uncommented to get adobe-ready versions
#ax.set_title(name)
ax.legend(**pyam.plotting.OUTSIDE_LEGEND['right'])
#ax.legend().remove()

# conform with previous plot
ax.set_ylabel('')

# harmonize ylim
ax.set_ylim(-4.5, 9.5)

fig.savefig('./files/fig2b2.pdf', bbox_inches='tight')

In [21]:
variables = [
    'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct', 
    'Emissions|CO2|AFOLU', 
]
name = 'OSCAR LULUCF Gap'
fig, ax = plt.subplots(figsize=(3, 9))
(
    df
    .filter(region='World', variable=variables)
    .filter(Temperature=categories_to_temp.values())
    .filter(year=range(2025, 2101, 25))
    .subtract(variables[0], variables[1], name=name, ignore_units='Mt CO2/yr')
    .convert_unit('Mt CO2/yr', to='Gt CO2/yr', factor=1e-3)
    .plot
    .box(color='thistle', by='Temperature', ax=ax, showfliers=False)
)
ax.set_title(name)
ax.set_ylim(None, 6)
ax.set_xlabel('')

# SI Table of Indirect Flux

In [22]:
def data_range(df, region, category, year, variable='AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Indirect'):
    category_kwarg = {'Category': category} if category.startswith('C') else {'Policy_category_name': category}
    data = (
        df
        .filter(region=region, variable=variable, year=year)
        .filter(**category_kwarg)
        .convert_unit('Mt CO2/yr', to='Gt CO2/yr', factor=1e-3)
        .compute.quantiles([0.25, 0.5, 0.75])
        ['value']
    )
    return data[1], data[0], data[2] # 0.5, 0.25, 0.75

regions = df.region
years = [2025, 2050, 2075, 2100]
categories = ['C1', 'C2', 'C3']

idx = pd.MultiIndex.from_product([regions, categories, years], names=["Region", "Category",  "Year"])
table = (
    pd.Series([
        '{:.2f} [{:.2f}, {:.2f}]'.format(*data_range(df, region, category, year)) for region, category, year in idx
    ], index=idx)
    .unstack(level=['Category', 'Year'])
)
table.head()

In [23]:
table.T.to_excel('./files/indirect_flux_values_test.xlsx', index=True)

# Table of median removals and emissions

In [24]:
categories = ['C1', 'P1b: Current policies']
regions = ['World']
years = [2050]
idx = pd.MultiIndex.from_product([regions, categories, years], names=["Region", "Category",  "Year"])

In [25]:
variables = [
    'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Direct',
    'AR6 Reanalysis|OSCARv3.2|Carbon Removal|Land|Indirect',
    'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Direct',
    'AR6 Reanalysis|OSCARv3.2|Emissions|CO2|AFOLU|Indirect',
]
for variable in variables:
    table = (
        pd.Series([
            '{:.2f} [{:.2f}, {:.2f}]'.format(*data_range(df, region, category, year, variable=variable)) for region, category, year in idx
        ], index=idx)
        .unstack(level=['Category', 'Year'])
    )
    print(variable)
    print(table)